<a href="https://colab.research.google.com/github/hasankere/Week5_Challenge_project/blob/main/ethiomart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset
from seqeval.metrics import classification_report
import sqlite3


In [ ]:
from google.colab import files

# Upload file
uploaded = files.upload()



Saving channels_to_crawls.csv to channels_to_crawls.csv


In [ ]:
import pandas as pd

# Load the uploaded file
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)

# Use the first column as the channel list
channel_list = channels_df[0].tolist()
print(channel_list)


['@ZemenExpress', '@nevacomputer', '@meneshayeofficial', '@ethio_brand_collection', '@Leyueqa', '@sinayelj', '@Shewabrand', '@helloomarketethiopia', '@modernshoppingcenter', '@qnashcom', '@Fashiontera', '@kuruwear', '@gebeyaadama', '@MerttEka', '@forfreemarket', '@classybrands', '@marakibrand', '@aradabrand2', '@marakisat2', '@belaclassic', '@AwasMart', '@qnashcom', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [ ]:
import pandas as pd
from telethon import TelegramClient
import nest_asyncio
import asyncio
from google.colab import files  # To upload files to Colab

nest_asyncio.apply()  # Allow nested event loops

# Telegram API credentials
api_id = '27904749'
api_hash = '5d9f2efaa785a2c4d04b5043088c2813'
phone = '+251912026082'

# Initialize Telegram Client
client = TelegramClient('session_name', api_id, api_hash)

async def fetch_messages(channel_name, limit=100):
    """Fetch messages from a specific Telegram channel."""
    messages = []
    try:
        async for message in client.iter_messages(channel_name, limit=limit):
            messages.append({
                'channel': channel_name,
                'sender': message.sender_id,
                'timestamp': message.date,
                'content': message.message,
                'media': message.media,
            })
    except Exception as e:
        print(f"Error fetching messages from {channel_name}: {e}")
    return messages

# Fetch messages from all channels
all_data = []

async def fetch_from_all_channels():
    """Fetch messages from all channels in the channel list."""
    for channel in channel_list:
        print(f"Fetching messages from {channel}...")
        channel_data = await fetch_messages(channel)
        all_data.extend(channel_data)

# Define the main coroutine
async def main():
    await client.start()  # Start the Telegram client
    await fetch_from_all_channels()  # Fetch messages from all channels

# Run the main coroutine
asyncio.run(main())

# Convert to DataFrame for easier handling
df = pd.DataFrame(all_data)

# Save to CSV in Colab's environment
output_path = '/content/telegram_messages.csv'  # Save to the current directory
df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

# If you'd like to download the file back to your local machine
files.download(output_path)


Fetching messages from @ZemenExpress...
Fetching messages from @nevacomputer...
Fetching messages from @meneshayeofficial...
Fetching messages from @ethio_brand_collection...
Fetching messages from @Leyueqa...
Fetching messages from @sinayelj...
Fetching messages from @Shewabrand...
Fetching messages from @helloomarketethiopia...
Fetching messages from @modernshoppingcenter...
Fetching messages from @qnashcom...
Fetching messages from @Fashiontera...
Fetching messages from @kuruwear...
Fetching messages from @gebeyaadama...
Fetching messages from @MerttEka...
Fetching messages from @forfreemarket...
Fetching messages from @classybrands...
Fetching messages from @marakibrand...
Fetching messages from @aradabrand2...
Fetching messages from @marakisat2...
Fetching messages from @belaclassic...
Fetching messages from @AwasMart...
Fetching messages from @qnashcom...
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import nltk

# Ensure 'punkt' tokenizer is downloaded
nltk.download('punkt')  # This is the standard tokenizer
nltk.download('stopwords')  # Optional: if you need stopwords for filtering

# Import necessary libraries
import pandas as pd
import string

# Function to preprocess Amharic text
def preprocess_amharic_text(text):
    # Convert the text to string if it's not already
    if isinstance(text, float):
        text = str(text)

    # Step 1: Normalize text - Remove diacritics (manual approach)
    normalized_text = ''.join([char for char in text if char not in ['፣', '፤', '፦', '፧', '፩', '፮', '፱', '፴', '፻']])

    # Step 2: Remove punctuation
    text_no_punctuation = ''.join([char for char in normalized_text if char not in string.punctuation])

    # Step 3: Tokenize text using nltk's word_tokenize
    try:
        tokens = nltk.word_tokenize(text_no_punctuation)
    except LookupError as e:
        print("Error during tokenization:", e)
        tokens = []  # If tokenization fails, return an empty list

    # Step 4: Convert to lowercase
    normalized_lower_text = ' '.join([word.lower() for word in tokens])

    return normalized_lower_text

# Load the CSV file with the filtered data (ensure it's uploaded to Colab first)
df = pd.read_csv('/content/telegram_messages.csv')

# Apply the preprocessing to the 'content' column (or the relevant column in your data)
df['processed_content'] = df['content'].apply(preprocess_amharic_text)

# Save the processed data to a new CSV file
output_path = '/content/processed_telegram_messages.csv'
df.to_csv(output_path, index=False)

# Download the file
from google.colab import files
files.download(output_path)

# Display the first few rows of the original and processed content
df[['content', 'processed_content']].head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Streaming output truncated to the last 5000 lines.
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error during tokenization: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,content,processed_content
0,NaN,
1,NaN,
2,NaN,
3,💥💥...................................💥💥\n\n📌 2...,
4,💥💥...................................💥💥\n\n📌 2...,


In [25]:
import pandas as pd
from google.colab import files
import string
import nltk

# Ensure the necessary NLTK data is downloaded
nltk.download('punkt')

# Function to preprocess and clean Amharic text
def preprocess_amharic_text(text):
    """Preprocess text: remove diacritics, punctuation, and tokenize."""
    # Convert the text to string if it's not already
    if isinstance(text, float):
        text = str(text)

    # Step 1: Normalize text - Remove diacritics (manual approach for common characters)
    normalized_text = ''.join([char for char in text if char not in ['፣', '፤', '፦', '፧', '፩', '፮', '፱', '፴', '፻']])

    # Step 2: Remove punctuation
    text_no_punctuation = ''.join([char for char in normalized_text if char not in string.punctuation])

    # Step 3: Tokenize text using nltk's word_tokenize
    try:
        tokens = nltk.word_tokenize(text_no_punctuation)
    except LookupError as e:
        print("Error during tokenization:", e)
        tokens = []  # If tokenization fails, return an empty list

    # Step 4: Convert to lowercase
    normalized_lower_text = ' '.join([word.lower() for word in tokens])

    return normalized_lower_text

# Load the CSV file (ensure the file has been uploaded to Colab)
df = pd.read_csv('/content/telegram_messages.csv')

# Step 4: Clean and structure the data
# We want to separate metadata and message content:
df['processed_content'] = df['content'].apply(preprocess_amharic_text)

# Ensure to separate metadata: sender_id, timestamp, and message content
df_cleaned = df[['sender', 'timestamp', 'processed_content']].copy()

# Step 5: Store preprocessed data in a structured format
# Save the cleaned data to a new CSV file
output_path = '/content/processed_telegram_messages.csv'
df_cleaned.to_csv(output_path, index=False)

# Optionally: Download the file for local use
files.download(output_path)

# Display the first few rows of the cleaned data
df_cleaned.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Streaming output truncated to the last 5000 lines.
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error during tokenization: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sender,timestamp,processed_content
0,-1001307493052,2025-01-16 06:29:40+00:00,
1,-1001307493052,2025-01-16 06:29:40+00:00,
2,-1001307493052,2025-01-16 06:29:40+00:00,
3,-1001307493052,2025-01-16 06:29:40+00:00,
4,-1001307493052,2025-01-16 06:28:10+00:00,


In [27]:
import pandas as pd

# Sample data, assuming you have a DataFrame from your telegram messages
df = pd.read_csv('/content/processed_telegram_messages.csv')

# Function to convert text into CoNLL format (this is a simple example)
def label_in_conll_format(text):
    """
    This function will label a text using a simple example for NER.
    You should replace this logic with your own manual or automated labeling mechanism.
    """
    # Check if the text is a valid string (not NaN)
    if not isinstance(text, str):
        text = ""  # If not a string, replace with empty string

    # Example predefined labels for tokens
    words = text.split()
    labeled_text = []

    for word in words:
        if word.lower() in ['john', 'doe']:  # Example for a person's name
            labeled_text.append(f"{word} B-PER")  # Label 'John' and 'Doe' as a person
        elif word.lower() in ['addis', 'ababa']:  # Example for a location
            labeled_text.append(f"{word} B-LOC")  # Label 'Addis' and 'Ababa' as a location
        else:
            labeled_text.append(f"{word} O")  # Label others as non-entity

    return "\n".join(labeled_text) + "\n"  # Return formatted text in CoNLL format

# Function to save labeled data into CoNLL format
def save_to_conll(df, output_path):
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Apply the labeling function to each content column
            labeled_text = label_in_conll_format(row['processed_content'])
            f.write(labeled_text)
            f.write("\n")  # Add an empty line after each sentence

# Call the function to save data in CoNLL format
output_path = '/content/labeled_telegram_data.conll'
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Data has been labeled and saved to: {output_path}")

# Optionally: Download the file for local use
from google.colab import files
files.download(output_path)


Data has been labeled and saved to: /content/labeled_telegram_data.conll


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>